In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
from PIL import Image
from IPython.display import HTML
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from matplotlib.animation import FuncAnimation

from IPython import display # manipulate notebook output

In [7]:

class DNA():
    def __init__(self, num_bps):
        
        # generate random DNA strand
        bps = ['AT','CG','TA','GC']
        self.full_strand = np.random.choice(bps, num_bps)
        self.s0 = None
        self.s1 = None
        
        
    def split(self):
        
        # denature DNA into two complimentary strands
        self.s0 = ''.join(pair[0] for pair in self.full_strand)
        self.s1 = ''.join(pair[1] for pair in self.full_strand)

        return self.s0, self.s1

    
    def fragment(self, strand, u_frag_length, stdev_frag_length):
        
        frags = []
        count = 0
        while count < len(strand):
            
            # fragment the strand as we go, sampling from a normal distribution for fragment length
            fl = np.array(np.random.normal(u_frag_length, stdev_frag_length)).astype(int)
            if fl <= 0: continue
            frags.append(strand[count:min(count+fl, len(strand))])
            count += fl
            
        return frags
    
    
    def filter_frags(self, frags, min_length, max_length):
        # only return fragments that are within a range of lengths
        return [strand for strand in frags if len(strand) >= min_length and len(strand) <= max_length]
        
    
    def replicate(self, strand, thresh, u_frag_length, stdev_frag_length, min_length, max_length):
        good_frags = []
        while len(good_frags) < thresh:
            # fragment strand
            fs = self.fragment(strand, u_length, stdev_length)
            # filter fragments
            fs_filt = self.filter_frags(fs, min_length, max_length)
            # record
            good_frags.extend(fs_filt)
                        
        return good_frags

        
        

In [8]:
# generate example DNA strand
t = DNA(10**2)

# denature into complimentary strands
s0, s1 = t.split()

# fragment the chosen strand and filter fragments based on length
u_length = 10
stdev_length = 3
min_length = u_length - stdev_length
max_length = u_length + stdev_length

thresh = 10e3 # number of good fragments required
frags = t.replicate(s0, thresh, u_length, stdev_length, min_length, max_length)

print(len(frags))

10008


In [9]:

# control the sparsity of the slide
A_ratio = 10 # this times as much empty space as dots
slide_dim = np.sqrt(thresh*A_ratio).astype(int)
slide = np.zeros((slide_dim, slide_dim, 3), dtype=np.uint8)

# generate random coordinates on flow cell
coords = set()    

while len(coords) < len(frags):
    x_temp = np.random.choice(np.arange(0,slide_dim))
    y_temp = np.random.choice(np.arange(0,slide_dim))
    
    coords.add((x_temp, y_temp))


In [10]:

# make synthetic visual data using frags ready for sequencing (exp_frags)

red = [255,0,0]
green = [0,255,0]
blue = [0,0,255]
yellow = [255,255,0]

color_dict = {'A':red, 'T':green, 'C':blue, 'G':yellow}

imgs = [] # store rgb images of slide

count = 0 # count the number of sequenced bases

prev_slide = slide.copy()
while count < max_length:
    for c,f in zip(coords,frags):
        if len(f) <= count: 
            slide[c[0], c[1],:] = [0,0,0]
            continue
        slide[c[0], c[1],:] = color_dict[f[count]]
    count += 1
    
    img = Image.fromarray(slide, 'RGB')
    imgs.append(img)


In [6]:

fig = plt.figure(figsize=(10,10))

def animate(frame):
    fig.clear()
    plt.imshow(imgs[frame])
    plt.title(f'seq step {frame}')
    plt.axis('off')

anim = FuncAnimation(fig, animate, frames=max_length, interval=200)
video = anim.to_html5_video()
plt.close()
display.HTML(video)  





In [ ]:
# try to assemble the fragments into the original strand
